# 2024 Election Data Playground

In [2]:
import geopandas as gpd
import numpy as np
from tqdm import tqdm 
import pandas as pd

In [ ]:
#load presidential results
pres_results = gpd.read_file("precincts-with-results.topojson")
pres_results["centroid"] = pres_results["geometry"].centroid
pres_results["district"] = None

#load districts data
file_name = "/Users/ryandonaldson/ryan_sandbox/election_data_2024/national_cong118_boundary/national_cong118_boundary/national_cong118_boundary.shp"
districts = gpd.read_file(file_name)


#sort the precints into districts
for i, row in tqdm(districts.iterrows(), total=len(districts)):
    precincts_in_district_bool = row["geometry"].contains(pres_results["centroid"])
    pres_results.loc[precincts_in_district_bool, "district"] = row["CONG118"]

#sum votes by district
pres_results_by_district = pres_results.groupby("district").agg({"votes_dem": "sum", "votes_rep": "sum",  "votes_total": "sum"})

#save intermediary results
pres_results_by_district.to_csv('pres_results_by_district.csv')

100%|██████████| 435/435 [02:49<00:00,  2.56it/s]


In [7]:
#load house and senate results
file_name = "/Users/ryandonaldson/ryan_sandbox/election_data_2024/election_results_house.csv"
house_results = gpd.read_file(file_name)

file_name = "/Users/ryandonaldson/ryan_sandbox/election_data_2024/election_results_senate.csv"
senate_results = gpd.read_file(file_name)

pres_results_by_district = pd.read_csv('pres_results_by_district.csv')

#sum precient data by district
districts["Presidential_Dem"] = None
districts["Presidential_Rep"] = None

#calculate voting percentage
pres_results_by_district["dem_perc_pres"] = pres_results_by_district["votes_dem"]/pres_results_by_district["votes_total"]
pres_results_by_district["rep_perc_pres"] = pres_results_by_district["votes_rep"]/pres_results_by_district["votes_total"]

#rename AT LARGE districts to 01 for joining
pres_results_by_district = pres_results_by_district.reset_index()
at_large_districts = pres_results_by_district["district"].str.contains("AT-LARGE")
districts = pres_results_by_district["district"].to_list()

def rename_func(x):
    if "AT-LARGE" in x:
        return(x[:-9] + "-01")
    else:
        return x
districts = list(map(rename_func,districts))
pres_results_by_district["district"] = districts

pres_results_by_district.set_index('district')

,index,votes_dem,votes_rep,votes_total,dem_perc_pres,rep_perc_pres
district,,,,,,
AR-01,0,74300,201263,280943,0.264466,0.716384
AR-02,1,133378,187500,328790,0.405663,0.570273
AR-03,2,114908,194657,318213,0.361104,0.611719
AR-04,3,81291,192704,279133,0.291227,0.690366
AZ-01,4,216232,230227,450724,0.479744,0.510794
...,...,...,...,...,...,...
WI-07,388,167604,266844,441113,0.379957,0.604933
WI-08,389,177232,247092,429848,0.412313,0.574836
WV-01,390,98066,259831,364662,0.268923,0.712526


In [ ]:
results_by_district = pres_results_by_district


#make a table for house dems
house_results["district"] = house_results["office_seat_name"].apply(lambda x: f"-{int(x[9:]):02}")
house_results["district"] = house_results["state_abbrev"] + house_results["district"]
house_results = house_results[house_results["cycle"] == "2024"]
house_results_dems = house_results[house_results["ballot_party"] == "DEM"]

#join pres results with house dems
results_by_district = results_by_district.set_index('district')
results_by_district = results_by_district.join(house_results_dems.set_index('district'), how='left', lsuffix='pres', rsuffix='house')
results_by_district = results_by_district.reset_index()

#simplify table
results_by_district = results_by_district[["district", "dem_perc_pres", "rep_perc_pres", "candidate_name", "percent", "ranked_choice_round"]]
results_by_district = results_by_district.sort_values(by=["district", "percent"])
results_by_district = results_by_district.drop_duplicates(keep='last', subset="district")

#remove duplicates
#results_by_district[results_by_district.duplicated(keep=False)]


#print table
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    # print(any(results_by_district.duplicated(subset="district")))
    print(results_by_district)

    district  dem_perc_pres  rep_perc_pres                 candidate_name  \
0      AR-01          0.264          0.716                  Rodney Govens   
1      AR-02          0.406          0.570                Marcus A. Jones   
2      AR-03          0.361          0.612                 Caitlin Draper   
3      AR-04          0.291          0.690                   Risie Howard   
4      AZ-01          0.480          0.511                     Amish Shah   
5      AZ-02          0.420          0.570                   Jonathan Nez   
6      AZ-03          0.692          0.294                Yassamin Ansari   
7      AZ-04          0.526          0.460                   Greg Stanton   
8      AZ-05          0.395          0.595              Katrina Schaffner   
9      AZ-06          0.491          0.498                  Kirsten Engel   
10     AZ-07          0.604          0.384                  Raúl Grijalva   
11     AZ-08          0.414          0.577                Gregory Whitten   

In [13]:
results_by_district.columns

Index(['district', 'dem_perc_pres', 'rep_perc_pres', 'candidate_name',
       'percent', 'ranked_choice_round'],
      dtype='object')

In [11]:
results_by_district["district"][results_by_district["district"].str.contains("AT_LARGE")]

Series([], Name: district, dtype: object)

In [ ]:
results_by_district = pres_results_by_district
for i, row in results_by_district.iterrows():
    state_bool = house_results["state_abbrev"] == row["district"][:2]

In [12]:
house_results.columns

Index(['id', 'race_id', 'state_abbrev', 'state', 'office_id', 'office_name',
       'office_seat_name', 'cycle', 'stage', 'special', 'party',
       'politician_id', 'candidate_id', 'candidate_name', 'ballot_party',
       'ranked_choice_round', 'votes', 'percent', 'unopposed', 'winner',
       'alt_result_text', 'source', 'district'],
      dtype='object')